# Utilização de Matrix Profile com STUMPY

In [ ]:
# Instalação de dependências não instaladas no colab por padrão
!pip install 'scipy>=1.5'
!pip install stumpy

In [ ]:
import pandas as pd
import stumpy

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.patches import Rectangle
import datetime as dt

plt.style.use('https://raw.githubusercontent.com/TDAmeritrade/stumpy/main/docs/stumpy.mplstyle')

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

## Busca por motifs

Base de dados gerada utilizando um modelo fuzzy para simular um gerador de vapor na Usina Abbott em Champaign, IL.

In [ ]:
# Leitura da base de dados
steam_df = pd.read_csv("https://zenodo.org/record/4273921/files/STUMPY_Basics_steamgen.csv?download=1")
steam_df.head()

,drum pressure,excess oxygen,water level,steam flow
0,320.08239,2.506774,0.032701,9.302970
1,321.71099,2.545908,0.284799,9.662621
2,320.91331,2.360562,0.203652,10.990955
3,325.00252,0.027054,0.326187,12.430107
4,326.65276,0.285649,0.753776,13.681666


**Visualizando o dataset**

In [ ]:
# Visualização da série temporal
fig = go.Figure()
fig.add_trace(go.Scatter(y=steam_df['steam flow'], name='steam flow'))
fig.show()

Identificando manualmente a existência de motifs

In [ ]:
m = 640 # tamanho da janela analisada

fig = make_subplots(rows=2, cols=1, shared_yaxes=True)

fig.add_trace(go.Scatter(y=steam_df['steam flow'], name='steam flow'), row=1, col=1)

# Construção do retangulos cinza onde está localizado os motifs
fig.add_vrect(x0=643, x1=643+m, line_width=0, fillcolor="grey", opacity=0.3,
              annotation_text="C1", annotation_position="top left", row=1, col=1)
fig.add_vrect(x0=8724, x1=8724+m, line_width=0, fillcolor="grey", opacity=0.3,
              annotation_text="C2", annotation_position="top left", row=1, col=1)

fig.add_trace(go.Scatter(y=steam_df['steam flow'][643:643+m], name='C1'), row=2, col=1)
fig.add_trace(go.Scatter(y=steam_df['steam flow'][8724:8724+m], name='C2'), row=2, col=1)

fig.update_yaxes(title='kg/s')
fig.show()

Procurando Motif usando STUMPY

In [ ]:
m = 640 # tamanho da janela analisada
mp = stumpy.stump(steam_df['steam flow'], m) # construição da Matrix Profile

A saída da função stump consiste em uma matrix com as seguintes colunas:

- 1º Matrix profile
- 2º Matrix profile index
- 3º Matrix profile index (left)
- 4º Matrix profile index (right)

In [ ]:
np.argsort(mp[:, 0]) # Indices das subsequências com vizinhos mais próximos

array([ 643, 8724, 8725, ..., 3865, 3863, 3864])

In [ ]:
# Seleciona o indice do primeira ocorrência do motif
motif_idx = np.argsort(mp[:, 0])[0]
print(f"The motif is located at index {motif_idx}")

The motif is located at index 643


In [ ]:
# Seleciona o indice da segunda ocorrência do motif
nearest_neighbor_idx = mp[motif_idx, 1]
print(f"The nearest neighbor is located at index {nearest_neighbor_idx}")

The nearest neighbor is located at index 8724


In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig.add_trace(go.Scatter(y=steam_df['steam flow'], name='steam flow'), row=1, col=1)

# Construção do retangulos cinza onde está localizado os motifs
fig.add_vrect(x0=643, x1=643+m, line_width=0, fillcolor="grey", opacity=0.3,
              annotation_text="C1", annotation_position="top left", row=1, col=1)
fig.add_vrect(x0=8724, x1=8724+m, line_width=0, fillcolor="grey", opacity=0.3,
              annotation_text="C2", annotation_position="top left", row=1, col=1)

fig.add_trace(go.Scatter(y=mp[:,0], name='Matrix Profile'), row=2, col=1)

# Construção das linhas verticais tracejadas
fig.add_vline(x=643, line_dash="dash", row=2, col=1)
fig.add_vline(x=8724, line_dash="dash", row=2, col=1)

fig.update_yaxes(title='kg/s', row=1, col=1)
fig.update_xaxes(title='index')
fig.show()

## Identificando anomalias

In [ ]:
# Seleciona o indice da subsequência com a menor similaridade com o restante
discord_idx = np.argsort(mp[:, 0])[-1]

print(f"Index localizado {discord_idx}")
print(f"Subsequencia vizinha mais próxima = {mp[discord_idx, 0]}")

Index localizado 3864
Subsequencia vizinha mais próxima = 23.47616836730196


In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig.add_trace(go.Scatter(y=steam_df['steam flow'], name='steam flow'), row=1, col=1)
fig.add_vrect(x0=discord_idx, x1=discord_idx+m, line_width=0, fillcolor="grey", opacity=0.3,
              row=1, col=1)

fig.add_trace(go.Scatter(y=mp[:,0], name='Matrix Profile'), row=2, col=1)
fig.add_vline(x=discord_idx, line_dash="dash", row=2, col=1)

fig.update_yaxes(title='kg/s', row=1, col=1)
fig.update_xaxes(title='index')
fig.show()

**Trocando o dataset**

Histórico de número de passageiros de taxi em Nova York ao longo de 75 dias no outono de 2014.


**Obs:** value  = Média de números de passageiro a cada meia hora 

In [ ]:
# Abrindo a base de dados com a série temporal
taxi_df = pd.read_csv("https://zenodo.org/record/4276428/files/STUMPY_Basics_Taxi.csv?download=1")

# Convertendo o tipo das variaveis
taxi_df['value'] = taxi_df['value'].astype(np.float64)
taxi_df['timestamp'] = pd.to_datetime(taxi_df['timestamp'])

taxi_df.head()

,timestamp,value
0,2014-10-01 00:00:00,12751.0
1,2014-10-01 00:30:00,8767.0
2,2014-10-01 01:00:00,7005.0
3,2014-10-01 01:30:00,5257.0
4,2014-10-01 02:00:00,4189.0


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(y=taxi_df['value']))
fig.update_yaxes(title='Média de passageiros')
fig.show()

In [ ]:
m = 48 # exatamente 1 dia
mp = stumpy.stump(taxi_df['value'], m=m) # calcula a Matrix Profile

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=taxi_df['timestamp'], y=mp[:,0], name='Matrix profile'))

# Adiciona notações indicando o que causou as anomalias
fig.add_annotation(x=taxi_df['timestamp'][540], y=1.6,
            text="Dia de Colombo",
            showarrow=True,
            arrowhead=2)
fig.add_annotation(x=taxi_df['timestamp'][1535], y=3.6,
            text="Horário de Verão",
            showarrow=True,
            arrowhead=2)
fig.add_annotation(x=taxi_df['timestamp'][2700], y=3,
            text="Dia de Ação de graça",
            showarrow=True,
            arrowhead=2)

fig.show()